In [ ]:
# # if your kernel do not contain the packages below, please run this cell 
# %conda install pandas numpy scipy scikit-learn

# # an alternative
# #%pip install pandas numpy scipy scikit-learn

In [1]:
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pandas as pd
from scipy.spatial.distance import cdist
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
file_path = 'E:/GitHub/Spring2020-Project3-group12/data'
os.chdir(file_path)

In [ ]:
# create train set
def dist(file):
    fname = list(file.keys())[-1]
    dat = file[fname]
    dist_mat = cdist(dat, dat, 'euclidean')
    return dist_mat[np.triu_indices(78, 1)].flatten()


label = pd.read_csv('../data/train_set/label.csv')
file_path = [os.path.join('../data/train_set/points/', str(i).zfill(4) + '.mat') for i in range(1, 2501)]

names = [f'featrue{n}' for n in range(3003)]
X = pd.DataFrame(list(map(dist, (loadmat(f) for f in file_path))), columns=names)
y = np.array(label['emotion_idx'])
groups = y

train_set = X
train_set['label'] = y

In [ ]:

'''
# PCA
pca = PCA()
pca.fit(X)
cum_var = pca.explained_variance_ratio_.cusum()
print(cum_var[cum_var<.999].shape)

# Grid Search RF
rf_params = {'n_estimators': [5, 10, 15, 30, 50],
             'max_depth': [5, 10, 15, 20]}

rf_grid = GridSearchCV(RandomForestClassifier(n_jobs=5, verbose=True),
                       rf_params, cv=5, return_train_score=True,
                       scoring='accuracy')
rf_grid.fit(X, y, groups=groups)
print("RandomForest Best Params", rf_grid.best_params_)
print("Score", rf_grid.best_score_)

# Grid Search SVM
svm_params = [{'kernel': ['linear', 'rbf'],
               'C': [1, 10, 50, 100],
               'gamma': [1e-4, 1e-3, 'auto'],
               'decision_function_shape': ['ovo', 'ovr']
               }]
svm_grid = GridSearchCV(SVC(), svm_params, cv=5, verbose=5, n_jobs=5, return_train_score=True, scoring='accuracy')
svm_grid.fit(X, y, groups=groups)
print("SVM best param", svm_grid.best_params_)
print("SVM scores", svm_grid.best_score_)


# Grid Search Logistic
lr_params = {'C': [.1, 1, 10]}
lr_grid = GridSearchCV(LogisticRegression(penalty='l2', max_iter=100),
                       lr_params, cv=5, n_jobs=5, return_train_score=True, verbose=10)
lr_grid.fit(X, y, groups=groups)
print(lr_grid.best_params_)

# Grid Search Ridge 
ridge_params = {'alpha': [1, 10, 50, 80, 85]
ridge_grid = GridSearchCV(Ridge,
                          ridge_grid, cv=5, n_jobs=5, return_train_score=True, verbose=10)
ridge_grid.fit(X, y, groups=groups)
print(ridge_grid.best_params_)
'''

In [ ]:
pca = PCA(n_components=128)

rf = RandomForestClassifier(n_estimators=800, max_depth=5, n_jobs=-1)
svm = SVC(kernel='linear', C=.001, decision_function_shape='ovo')
svm_bag = BaggingClassifier(svm, n_estimators=120, n_jobs=-1)
ridge = RidgeClassifier(alpha=85)
lr = LogisticRegression(n_jobs=-1)
voting_est = [('svm', svm), ('ridge', ridge), ('logi', lr)]
voting = VotingClassifier(estimators=voting_est, voting='hard', n_jobs=-1)

estimators = {'RandomForest': rf,
              'SVM': svm,
              'SVM with Bagging': svm_bag,
              'Ridge': ridge,
              'Logistic Regression': lr,
              'Voting(SVM+Ridge+LR)': voting}

In [ ]:
# Cross_validation Score
for name, estimator in estimators.items():
    train_scores = []
    test_scores = []
    times = []
    for i in range(5):
        s = time.time()
        X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=.2, stratify=y)
        X_train_pca = pca.fit_transform(X_train)
        X_validation_pca = pca.transform(X_validation)

        # fitting model
        estimator.fit(X_train_pca, y_train)
        test_pred = estimator.predict(X_validation_pca)

        # return train model time
        times.append(time.time() - s)

        train_pred = estimator.predict(X_train_pca)

        train_score = estimator.score(X_train_pca, y_train)
        test_score = estimator.score(X_validation_pca, y_validation)

        train_scores.append(train_score)
        test_scores.append(test_score)

    print(f'{name}, Mean Train score: {train_scores.mean():.4f}')
    print(f'{name}, Mean Test score: {test_scores.mean():.4f}')
    print(f'{name}, Mean Time spent: {times.mean():.4f} seconds')
